In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from pytube import YouTube
import os
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
import requests

import pickle
import sys
from pathlib import Path
from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM
from tensorflow.keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Conv1D, Dense, concatenate
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox
import pysrt

In [ ]:
SAVE_PATH = "C:\\Users\\leena\\Desktop\\Python Projects\\emotion-from-tweet\\notebooks"
filename = "video2.mp4"

In [ ]:
sys.path.append(Path(os.path.join(os.path.abspath(''), '../')).resolve().as_posix())
tokenizer_path = Path('../datasets/sentiment_analysis/tokenizer.pickle').resolve()
with tokenizer_path.open('rb') as file:
    tokenizer = pickle.load(file)

input_dim = min(tokenizer.num_words, len(tokenizer.word_index) + 1)
num_classes = 4
embedding_dim = 500
input_length = 100
lstm_units = 128
lstm_dropout = 0.1
recurrent_dropout = 0.1
spatial_dropout=0.2
filters=64
kernel_size=3

input_layer = Input(shape=(input_length,))
output_layer = Embedding(
  input_dim=input_dim,
  output_dim=embedding_dim,
  input_shape=(input_length,)
)(input_layer)

output_layer = SpatialDropout1D(spatial_dropout)(output_layer)

output_layer = Bidirectional(
LSTM(lstm_units, return_sequences=True,
     dropout=lstm_dropout, recurrent_dropout=recurrent_dropout)
)(output_layer)
output_layer = Conv1D(filters, kernel_size=kernel_size, padding='valid',
                    kernel_initializer='glorot_uniform')(output_layer)

avg_pool = GlobalAveragePooling1D()(output_layer)
max_pool = GlobalMaxPooling1D()(output_layer)
output_layer = concatenate([avg_pool, max_pool])

output_layer = Dense(num_classes, activation='softmax')(output_layer)

model = Model(input_layer, output_layer)

model_weights_path = Path('../models/emotion_recognition/model_weights.h5').resolve()
model.load_weights(model_weights_path.as_posix())
encoder_path = Path('../models/emotion_recognition/encoder.pickle').resolve()
with encoder_path.open('rb') as file:
    encoder = pickle.load(file)

In [ ]:
def downloadVideo(link):
    yt = YouTube(link)
    stream = yt.streams.first()
    out_file = stream.download(SAVE_PATH)
    os.rename(out_file, filename)
    print("Download successful - " + str(out_file))

In [ ]:
def generateCaptions(link,lang):
    yt = YouTube(link)
    print(yt.captions.all())
    en_caption = yt.captions.get_by_language_code(lang)
    en_caption_convert_to_srt =(en_caption.generate_srt_captions())
    text_file = open("caption.srt", "w")
    text_file.write(en_caption_convert_to_srt)
    text_file.close()

In [ ]:
def parseSubtitles():
    sub = pysrt.open("caption.srt")
    global dialogs
    global start
    global end
    dialogs = []
    start = []
    end = []
    for obj in sub:
        dialogs.append(obj.text)
        start.append(obj.start.seconds)
        end.append(obj.end.seconds)

In [ ]:
def splitVideo():
    for i in range(len(dialogs)):
        ffmpeg_extract_subclip(filename, start[i],end[i], targetname=dialogs[i][:8]+".mp4")

In [ ]:
def fetchNews(country,category):
    response = requests.get("http://newsapi.org/v2/top-headlines?country="+country+"&category="+category+"&apiKey=469e3b5dc7b44bf6ae90867fe0e8fd5c")
    data = response.json()
    global articles
    global news
    news = []
    articles = data['articles']
    for article in articles:
        print(article['title'])
        title_parts = article['title'].split(" - ")
        text = title_parts[0] + article['description']
        news.append(text)

In [ ]:
def calculateEmotion(selected):
    sequences = [text.split() for text in selected]
    list_tokenized = tokenizer.texts_to_sequences(sequences)
    x_data = pad_sequences(list_tokenized, maxlen=100)

    y_pred = model.predict(x_data)
    emo = dict()
    for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
        print(encoder.classes_[index] + ": " + str(value))
        emo[encoder.classes_[index]]=value

In [ ]:
def insert_newlines(string, every=64):
    return '\n'.join(string[i:i+every] for i in range(0, len(string), every))

In [ ]:
def generateFinalOutput(videoClip,text, dialog):
    clip = VideoFileClip(videoClip)  
    text = insert_newlines(text)
    dialog = insert_newlines(dialog)
    time = clip.duration
    txt_clip = TextClip(text, fontsize = 22, color = 'white') 
    txt_clip2 = TextClip(dialog, fontsize = 15, color = 'yellow')
    txt_clip = txt_clip.set_pos('top').set_duration(time) 
    txt_clip2 = txt_clip2.set_pos('bottom').set_duration(time) 
    video = CompositeVideoClip([clip, txt_clip,txt_clip2])  
    video.write_videofile('Meme_'+videoClip[:8] + text[:8]+ '.mp4')

In [ ]:
def getMeme(dialogs,news):
    movie_line = 0
    pairs = dict()
    global_diff = 6
    for j in range(len(dialogs)):
        selected = dialogs[j]
        sequences = [text.split() for text in selected]
        list_tokenized = tokenizer.texts_to_sequences(sequences)
        x_data = pad_sequences(list_tokenized, maxlen=100)

        y_pred = model.predict(x_data)
        emo = dict()
        for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
            print(encoder.classes_[index] + ": " + str(value))
            emo[encoder.classes_[index]]=value

        min_diff = 3
        news_line = 0
        for i in range(len(news)):
            cleaned_data = news[i]
            sequences = [text.split() for text in cleaned_data]
            list_tokenized = tokenizer.texts_to_sequences(sequences)
            x_data = pad_sequences(list_tokenized, maxlen=100)

            y_pred = model.predict(x_data)

            cur_diff = 0
            for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
                cur_diff += abs(emo[encoder.classes_[index]] - value)

            if cur_diff < min_diff:
                news_line = i
                min_diff = cur_diff

        if min_diff < global_diff:
            movie_line = j
            global_diff = min_diff
            
        disp = articles[news_line]['title'] + " : " + dialogs[j]
        print(articles[news_line]['title'] + " : " + dialogs[j] )
        text = insert_newlines(disp,40)
        generateFinalOutput(dialogs[j][:8] + '.mp4', articles[news_line]['title'], dialogs[j] )
        pairs[j] = news_line
    
    print("Final Output --> "+ news[pairs[movie_line]] + " : " + dialogs[movie_line])

In [ ]:
def getDialogMeme(selected,file):
    sequences = [text.split() for text in selected]
    list_tokenized = tokenizer.texts_to_sequences(sequences)
    x_data = pad_sequences(list_tokenized, maxlen=100)

    y_pred = model.predict(x_data)
    emo = dict()
    for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
        print(encoder.classes_[index] + ": " + str(value))
        emo[encoder.classes_[index]]=value
        
    min_diff = 5
    ans = 0
    for i in range(len(news)):
        cleaned_data = news[i]
        sequences = [text.split() for text in cleaned_data]
        list_tokenized = tokenizer.texts_to_sequences(sequences)
        x_data = pad_sequences(list_tokenized, maxlen=100)

        y_pred = model.predict(x_data)

        cur_diff = 0
        for index, value in enumerate(np.sum(y_pred, axis=0) / len(y_pred)):
            cur_diff += abs(emo[encoder.classes_[index]] - value)

        if cur_diff < min_diff:
            ans = i
            min_diff = cur_diff
            
    print(news[ans] + " : " + selected)
    generateFinalOutput(file, news[ans], selected )

In [ ]:
def demoInfinityWarTrailer():
    link = "https://www.youtube.com/watch?v=6ZfuNTqbHE8&t"
    print("Downloading video...")
    downloadVideo(link)
    subs = "There was an idea, to bring together a group of remarkable people, to see if we could become something more. So when they needed us we could fight the battles that they never could. In time, you'll know what it's like to lose. To feel so desperately that you're right, that you fail all the same. Dread it, Run from it. Destiny still arrives. Evacuate the city. Engage all defences and get this man a shield. Fun isn't something one considers the balance in the universe. But this does put a smile on my face. Who the hell are you guys?"
    global dialogs
    global start
    global end
    dialogs = subs.split(". ")
    start=[4,20,45,51,61,67,77,80,97,104,135]
    end=[18,29,50,56,66,71,79,86,102,111,139]
    print("Splitting video...")
    splitVideo()
    print("Fetching news...")
    fetchNews('in','entertainment')
    print("Generating all Memes...")
    getMeme(dialogs,news)

In [ ]:
textYoutubeLink = widgets.Text(
    placeholder='Enter youtube video link',
    description='Youtube: ',
    disabled=False
)

def clickedDownload(arg): 
    downloadVideo(textYoutubeLink.value)
    
button_download = Button(description = 'Download Video')   
button_download.on_click(clickedDownload)

def clickedSplit(arg):
    generateCaptions(textYoutubeLink.value,'en')
    parseSubtitles()
    splitVideo()
    print("Video splitting successful!")
    
button_split = Button(description = 'Split Video')   
button_split.on_click(clickedSplit)

selCountry = widgets.Select(options=['ae','ar','at','au','be','bg','br','ca','ch','cn','co','cu','cz','de','eg','fr','gb','gr','hk','hu','id','ie','il','in','it','jp','kr','lt','lv','ma','mx','my','ng','nl','no','nz','ph','pl','pt','ro','rs','ru','sa','se','sg','si','sk','th','tr','tw','ua','us','ve','za'],value = 'in', rows = 1, description = 'Country: ')
selCategory = widgets.Select(options=['business','entertainment','general','health','science','sports','technology'],value = 'entertainment', rows = 7, description = 'Category: ')

def clickedNews(arg): 
    fetchNews(selCountry.value,selCategory.value)
    
button_news = Button(description = 'Fetch News')   
button_news.on_click(clickedNews)

def clickedFull(arg):
    demoInfinityWarTrailer()
    
button_demo = Button(description = 'Generate All Memes')   
button_demo.on_click(clickedFull)

textFilename = widgets.Text(
    value=filename,
    placeholder='Enter file name',
    description='File Name: ',
    disabled=False
)

dialog_text = "But this does put a smile on my face."
def clickedOneMeme(arg): 
    getDialogMeme(txtml.value,textFilename.value)

txtml = widgets.Textarea(value=dialog_text,placeholder = "Type the dialog", description = "Enter dialog: ")
button_meme = Button(description = 'Generate Meme')   
button_meme.on_click(clickedOneMeme)

txtCustom = widgets.Textarea(placeholder = "Custom Meme line", description = "Enter line: ")
def clickedCustom(arg): 
    generateFinalOutput(textFilename.value, txtCustom.value, txtml.value )
    
button_custom = Button(description = 'Custom Meme')   
button_custom.on_click(clickedCustom)


In [ ]:
filename = "video3.mp4"
print("Meme Builder Dashboard")
display(button_demo)
display(HBox([textYoutubeLink,button_download,button_split]))
display(selCategory)
display(selCountry)
display(button_news)
#display(txtml)
display(HBox([txtml,textFilename,button_meme]))
display(HBox([txtCustom,button_custom]))